In [3]:
!git clone https://github.com/lhldanh/Autoencoder-based-unsupervised-feature-learning-system.git
%cd Autoencoder-based-unsupervised-feature-learning-system/

Cloning into 'Autoencoder-based-unsupervised-feature-learning-system'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 211 (delta 120), reused 145 (delta 64), pack-reused 0 (from 0)
Receiving objects: 100% (211/211), 116.83 KiB | 2.66 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/kaggle/working/Autoencoder-based-unsupervised-feature-learning-system


In [4]:
%mkdir -p build
%mkdir -p weights
%mkdir -p data
!wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz -O data/cifar-10-binary.tar.gz
!tar -xzvf data/cifar-10-binary.tar.gz -C data

--2025-12-14 14:01:33--  https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170052171 (162M) [application/x-gzip]
Saving to: ‘data/cifar-10-binary.tar.gz’

data/cifar-10-binar 100%[===================>] 162.17M  55.3MB/s    in 2.9s    

2025-12-14 14:01:36 (55.3 MB/s) - ‘data/cifar-10-binary.tar.gz’ saved [170052171/170052171]

cifar-10-batches-bin/
cifar-10-batches-bin/data_batch_1.bin
cifar-10-batches-bin/batches.meta.txt
cifar-10-batches-bin/data_batch_3.bin
cifar-10-batches-bin/data_batch_4.bin
cifar-10-batches-bin/test_batch.bin
cifar-10-batches-bin/readme.html
cifar-10-batches-bin/data_batch_5.bin
cifar-10-batches-bin/data_batch_2.bin


In [6]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


## train (phase 3 full image)

In [9]:
!nvcc -arch=sm_75 -o build/train_gpu_optimize src/train_gpu_optimize.cu src/cifar10_dataset.cpp -I include/
# !nvcc src/train_gpu_optimize.cu src/cifar10_dataset.cpp -o build/train_gpu_optimize -O3 -use_fast_math -arch=sm_75 -lcuda -lcudart -I include/

In [10]:
%cd build/
!./train_gpu_optimize
%cd ..

/kaggle/working/Autoencoder-based-unsupervised-feature-learning-system/build
=== CUDA Autoencoder (Fused Backward Kernels) ===

GPU: Tesla T4
SMs: 40

--- Loading CIFAR-10 Dataset ---
Loaded batch: ../data/cifar-10-batches-bin/data_batch_1.bin | Current Total: 10000
Loaded batch: ../data/cifar-10-batches-bin/data_batch_2.bin | Current Total: 20000
Loaded batch: ../data/cifar-10-batches-bin/data_batch_3.bin | Current Total: 30000
Loaded batch: ../data/cifar-10-batches-bin/data_batch_4.bin | Current Total: 40000
Loaded batch: ../data/cifar-10-batches-bin/data_batch_5.bin | Current Total: 50000
Loaded batch: ../data/cifar-10-batches-bin/test_batch.bin | Current Total: 10000
Successfully loaded 50000 train images and 10000 test images.
Images: 50000

Memory: 1779 MB

Training: 20 epochs, 781 batches

Epoch 1/20 | Loss: 0.109956 | Time: 119.41s | 419 img/s
Epoch 2/20 | Loss: 0.085264 | Time: 121.62s | 411 img/s
Epoch 3/20 | Loss: 0.083649 | Time: 121.77s | 410 img/s
Epoch 4/20 | Loss: 0.082